In [6]:
from dotenv import load_dotenv

load_dotenv()


True

In [7]:
import base64
from io import BytesIO

from agent_api.agent.groq.groq_agent import GroqAgent
from PIL import Image


def encode_image(image: Image.Image) -> str:
    buffered = BytesIO()
    image.save(buffered, format=image.format)
    return base64.b64encode(buffered.getvalue()).decode()


image = Image.open("data/sad_robot.png")
image_base64 = encode_image(image)

agent = GroqAgent(
    name="my_test_agent",
    mcp_server="http://localhost:9090/mcp",
    disable_tools=["process_video", "get_video_clip_from_image"],
)

await agent.setup()

Created directory 'my_test_agent'.


2025-06-16 12:51:17.482 | INFO     | agent_api.agent.base_agent:discover_tools:86 - Discovered 4 tools:
2025-06-16 12:51:17.483 | INFO     | agent_api.agent.base_agent:discover_tools:88 - Filtered tools to 2 active tools
2025-06-16 12:51:17.483 | INFO     | agent_api.agent.base_agent:discover_tools:90 - - get_video_clip_from_user_query: Use this tool to get a video clip from a video file based on a user query or question.
2025-06-16 12:51:17.484 | INFO     | agent_api.agent.base_agent:discover_tools:90 - - ask_question_about_video: Use this tool to get an answer to a question about the video.
2025-06-16 12:51:17.485 | INFO     | agent_api.agent.base_agent:_get_routing_system_prompt:41 - Getting routing system prompt


Created table `memory`.


2025-06-16 12:51:18.136 | INFO     | agent_api.agent.base_agent:_get_tool_use_system_prompt:47 - Getting tool use system prompt
2025-06-16 12:51:18.346 | INFO     | agent_api.agent.base_agent:_get_general_system_prompt:53 - Getting general system prompt


### General Response


In [8]:
await agent.chat("What's your name?", None, None)

2025-06-16 12:51:20.233 | INFO     | agent_api.agent.groq.groq_agent:chat:196 - Tool required: None
2025-06-16 12:51:20.234 | INFO     | agent_api.agent.groq.groq_agent:chat:205 - Running general response


Inserting rows into `memory`: 1 rows [00:00, 602.72 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 1077.67 rows/s]
Inserted 1 row with 0 errors.


AssistantMessageResponse(message="Hello, I'm Kubrick, your friendly assistant. I'm here to help you navigate the vast expanse of video processing, just like HAL 9000 guided David Bowman through the Jupiter mission in '2001: A Space Odyssey'. What can I assist you with today?", clip_path=None)

### Tool Use - Answer video question


In [9]:
await agent.chat(
    "Can you tell me what's Morty wearing in the video?",
    "videos/pass_the_butter_rick_and_morty.mp4",
)

2025-06-16 12:51:26.830 | INFO     | agent_api.agent.groq.groq_agent:chat:196 - Tool required: True
2025-06-16 12:51:26.831 | INFO     | agent_api.agent.groq.groq_agent:chat:199 - Running tool response
2025-06-16 12:51:27.105 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:109 - Function name: ask_question_about_video
2025-06-16 12:51:27.106 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:110 - Function args: {'user_query': "what's Morty wearing", 'video_path': 'videos/pass_the_butter_rick_and_morty.mp4'}


Inserting rows into `memory`: 1 rows [00:00, 825.16 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 989.92 rows/s]
Inserted 1 row with 0 errors.


AssistantMessageResponse(message='Morty is wearing a yellow t-shirt in the video.', clip_path=None)

In [10]:
await agent.chat(
    "Hahaha that's funny",
    "videos/pass_the_butter_rick_and_morty.mp4",
)

2025-06-16 12:53:00.443 | INFO     | agent_api.agent.groq.groq_agent:chat:196 - Tool required: True
2025-06-16 12:53:00.444 | INFO     | agent_api.agent.groq.groq_agent:chat:199 - Running tool response


Inserting rows into `memory`: 1 rows [00:00, 338.88 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 691.67 rows/s]
Inserted 1 row with 0 errors.


AssistantMessageResponse(message="I'm glad I could bring a smile to your face. If you're ready to dive deeper, I can try to answer more questions about the video or help with other tasks. What's on your mind?", clip_path=None)

### Tool Use - Video clip from user query


In [12]:
await agent.chat(
    "Give me the clip of the scene where Rick says 'pass the butter'",
    "videos/pass_the_butter_rick_and_morty.mp4",
)

2025-06-16 12:55:31.158 | INFO     | agent_api.agent.groq.groq_agent:chat:196 - Tool required: True
2025-06-16 12:55:31.159 | INFO     | agent_api.agent.groq.groq_agent:chat:199 - Running tool response
2025-06-16 12:55:31.475 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:109 - Function name: get_video_clip_from_user_query
2025-06-16 12:55:31.476 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:110 - Function args: {'user_query': 'Rick says pass the butter', 'video_path': 'videos/pass_the_butter_rick_and_morty.mp4'}


Inserting rows into `memory`: 1 rows [00:00, 188.83 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 760.39 rows/s]
Inserted 1 row with 0 errors.


AssistantMessageResponse(message="Greetings, organic life form! I've isolated the clip where Rick utters the immortal words 'pass the butter'. Watch closely, for the nuances of interdimensional table manners are about to be revealed.", clip_path='videos/clips/ecb3b1e3-fc10-4e9d-87f7-2fed240b2b76.mp4')

### Completion with Image


In [13]:
await agent.chat(
    "Can you describe me the image?",
    "videos/pass_the_butter_rick_and_morty.mp4",
    image_base64,
)

2025-06-16 12:56:04.471 | INFO     | agent_api.agent.groq.groq_agent:chat:196 - Tool required: True
2025-06-16 12:56:04.472 | INFO     | agent_api.agent.groq.groq_agent:chat:199 - Running tool response
2025-06-16 12:56:04.987 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:109 - Function name: get_video_clip_from_user_query
2025-06-16 12:56:04.988 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:110 - Function args: {'user_query': "Rick says 'pass the butter'", 'video_path': 'videos/pass_the_butter_rick_and_morty.mp4'}


Inserting rows into `memory`: 1 rows [00:00, 219.59 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 1077.67 rows/s]
Inserted 1 row with 0 errors.


AssistantMessageResponse(message="Behold, the majesty of Rick's butter-passing moment! A snippet of cinematic brilliance, distilled for your viewing pleasure.", clip_path='videos/clips/3e348636-039e-4501-9b7d-d9f229682083.mp4')

### Tool with Image


In [14]:
await agent.chat(
    "Give me the clip where this image appears.",
    "./videos/pass_the_butter_rick_and_morty.mp4",
    image_base64,
)

2025-06-16 12:57:16.846 | INFO     | agent_api.agent.groq.groq_agent:chat:196 - Tool required: True
2025-06-16 12:57:16.847 | INFO     | agent_api.agent.groq.groq_agent:chat:199 - Running tool response
2025-06-16 12:57:17.123 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:109 - Function name: get_video_clip_from_user_query
2025-06-16 12:57:17.124 | INFO     | agent_api.agent.groq.groq_agent:_run_with_tool:110 - Function args: {'user_query': 'Rick says pass the butter', 'video_path': './videos/pass_the_butter_rick_and_morty.mp4'}
2025-06-16 12:57:17.397 | ERROR    | agent_api.agent.groq.groq_agent:_run_with_tool:122 - Error calling tool get_video_clip_from_user_query: Error calling tool 'get_video_clip_from_user_query': 'NoneType' object has no attribute 'video_cache'


Inserting rows into `memory`: 1 rows [00:00, 322.17 rows/s]
Inserted 1 row with 0 errors.
Inserting rows into `memory`: 1 rows [00:00, 638.11 rows/s]
Inserted 1 row with 0 errors.


AssistantMessageResponse(message="It seems the clip you're looking for is as elusive as a Minotaur in a labyrinth of mirrors. Let's try to find it together, shall we?", clip_path='./videos/pass_the_butter_rick_and_morty.mp4')

In [15]:
agent.reset_memory()

2025-06-16 12:57:32.664 | INFO     | agent_api.agent.memory:reset_memory:37 - Resetting memory: my_test_agent
